In [16]:
import numpy as np

def TInvPower(A, k, x0=None, mu0=None, tol=1e-12, max_iter=50, verbose=True):
    w = A.shape[0]
    if x0 is None:
        x = smallest_sv(A)
    else:
        x = x0
    if mu0 is None:
        mu = x.T @ A @ x
    else:
        mu = mu0
    if verbose:
        print("non-sparse mu: ", mu)
        print("non-sparse x: ", x)
    update_size = np.inf 
    it = 0
    while update_size>tol and it<max_iter:
        x /= np.linalg.norm(x)
        try:
            y = np.linalg.solve(A - mu * np.eye(w), x) # Rayleigh iteration
            # y = np.linalg.solve(A, x) # pure inverse iteration
        except:
            print("Exiting early due to singular matrix")
            return x, mu
        y /= np.linalg.norm(y)
        if verbose:
            print("non-sparse y:", y)
        inds = np.argpartition(np.abs(y), -k)[-k:] # indices of largest absolute entries
        #y = keep_inds(y, inds)
        y = solve(A, inds)
        y /= np.linalg.norm(y)
        #mu = y.T @ A @ y # comment out to fix mu
        update_size = min(np.linalg.norm(y - x), np.linalg.norm(y + x))/np.linalg.norm(y)
        if verbose:
            print("x:", x, "y:", y, "mu:", mu, "update_size:", update_size)
        x = y
        it += 1
    return x, mu, it

def keep_inds(vector, inds): # set all but inds of vector to 0
    temp = vector*0
    temp[inds] = vector[inds]
    return temp

def smallest_sv(A):
    U, Sigma, V = np.linalg.svd(A, full_matrices=True)
    V = V.transpose()  # since numpy SVD returns the transpose
    return V[:, -1] # smallest singular vector

def solve(A, inds):
    w = A.shape[1]
    x = np.zeros(shape=(w,))
    x[np.ix_(inds)] = smallest_sv(A[np.ix_(inds, inds)]) # work on submatrix with inds
    return x

In [20]:
def generate_matrix(eigs, k, noise=0): # note: first eig should be smallest one
    w = len(eigs)
    A = np.zeros((w, w))
    u_list = []
    for i in range(w):
        u_list.append(np.random.normal(0, 1, w))
    u_list[0][k:] = 0
    U = np.vstack(u_list).T
    Q, R = np.linalg.qr(U) # orthogonalize u_list
    for i in range(w):
        A += eigs[i] * np.outer(Q[:, i], Q[:, i])
    A += np.random.normal(0, noise, (w, w))
    return A, Q

np.random.seed(1) # fix random seed for reproducibility

eigs = [0.01, 0.02, 0.03, 1, 2, 3] + [0.011]*5
k = 3
A, Q = generate_matrix(eigs, k, noise=1e-4) #2e-4
x, mu, it = TInvPower(A, k, mu0=0, verbose=True)
#print("A:", A)
#print("Q:", Q)
U, Sigma, V = np.linalg.svd(A, full_matrices=True)
print("Singular values of A:", Sigma)
print("Smallest singular vector of A:", smallest_sv(A))
print(f"Ended in {it} iterations")
print("x:", x, "mu:", mu, "error:", min(np.linalg.norm(x-Q[:, 0]), np.linalg.norm(x+Q[:, 0])))
print("True x:", Q[:, 0])
print("True Rayleigh quotient after noise:", Q[:, 0].T @ A @ Q[:, 0]) 
print("Residual norm:", np.linalg.norm(A @ x))
print("True x residual norm:", np.linalg.norm(A@ Q[:, 0]))

non-sparse mu:  0
non-sparse x:  [-0.85759798  0.25572267  0.40874436  0.00485466  0.00532242  0.07028402
  0.02969083  0.02150184  0.13733047 -0.06502165 -0.05134775]
non-sparse y: [-8.61337465e-01  2.46885033e-01  4.07441157e-01  5.61173956e-03
 -3.56630604e-05  6.89082164e-02  3.05949739e-02  2.89735774e-02
  1.35787930e-01 -5.95870491e-02 -5.09207410e-02]
x: [-0.85759798  0.25572267  0.40874436  0.00485466  0.00532242  0.07028402
  0.02969083  0.02150184  0.13733047 -0.06502165 -0.05134775] y: [ 0.89559468 -0.33626268 -0.29126891  0.          0.          0.
  0.          0.          0.          0.          0.        ] mu: 0 update_size: 0.2319280082190977
non-sparse y: [ 0.89833802 -0.32048513 -0.29991395 -0.00222271  0.00444631 -0.00732527
 -0.00422438 -0.00858718 -0.0121569  -0.00176439  0.00297476]
x: [ 0.89559468 -0.33626268 -0.29126891  0.          0.          0.
  0.          0.          0.          0.          0.        ] y: [ 0.89559468 -0.33626268 -0.29126891  0.          